In [1]:
import pickle
import pandas as pd
import numpy as np
import json
import re
import time
import ast

### 1. Load Dataset

In [2]:
dataset_df = pd.read_csv('master_datatset1.csv', index_col=0)
dataset_df = dataset_df.reset_index(drop=True)
dataset_df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,5139950212,KR,1618772961239,919,420,11,13,11.8.370.4668,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,5139907683,KR,1618770659706,1709,420,11,13,11.8.370.4668,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,5139845675,KR,1618767685265,1059,420,11,13,11.8.370.4668,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,5139893269,KR,1618766293131,945,420,11,13,11.8.370.4668,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,5139729965,KR,1618763463947,1932,420,11,13,11.8.370.4668,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5110788940,KR,1617522551541,1671,420,11,13,11.7.366.7612,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
19996,5109444003,KR,1617462955764,1276,420,11,13,11.7.366.7612,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
19997,5109329175,KR,1617461700741,1001,420,11,13,11.7.366.7612,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
19998,5109342286,KR,1617460222797,911,420,11,13,11.7.366.7612,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [3]:
dataset_df['gameCreation']

0        1618772961239
1        1618770659706
2        1618767685265
3        1618766293131
4        1618763463947
             ...      
19995    1617522551541
19996    1617462955764
19997    1617461700741
19998    1617460222797
19999    1617458647445
Name: gameCreation, Length: 20000, dtype: int64

In [33]:
print('Game version:', dataset_df['gameVersion'].unique())
print('QueueId:', dataset_df['queueId'].unique(), end='\n\n')  # 420: solo-rank, 430: blind, 440: free-rank
dataset_df.reset_index()

Game version: ['11.2.353.8505' '11.1.352.5559' '10.25.350.1724' '11.3.357.5376'
 '11.1.351.8352' '11.3.356.7268' '10.25.348.1797' '10.24.345.8390'
 '10.22.341.643' '10.21.339.2173' '10.20.338.336' '10.20.337.6669'
 '10.19.336.4199' '10.23.343.2581' '10.18.333.8889' '10.16.330.9186'
 '10.15.330.344' '10.15.328.8759' '10.14.327.8505']
QueueId: [ 420  440  900  450  430 1300  840  700  830]



,index,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,0,4944639329,KR,1611410105916,2182,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,1,4944514774,KR,1611407106346,1964,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,2,4944416028,KR,1611405333308,1392,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,3,4943531794,KR,1611388855532,961,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,4,4943391359,KR,1611386148030,1992,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3186,3186,4960395587,KR,1611932557988,615,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3187,3187,4960342119,KR,1611929791085,1276,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3188,3188,4960209671,KR,1611929193082,196,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3189,3189,4960098237,KR,1611926809769,1335,420,11,13,11.2.353.8505,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [34]:
max(dataset_df['gameCreation'])

1613386276003

In [35]:
ast.literal_eval(dataset_df['teams'][0])

[{'teamId': 100,
  'win': 'Win',
  'firstBlood': False,
  'firstTower': False,
  'firstInhibitor': True,
  'firstBaron': False,
  'firstDragon': True,
  'firstRiftHerald': False,
  'towerKills': 7,
  'inhibitorKills': 1,
  'baronKills': 1,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 82, 'pickTurn': 1},
   {'championId': 234, 'pickTurn': 2},
   {'championId': 523, 'pickTurn': 3},
   {'championId': 35, 'pickTurn': 4},
   {'championId': 55, 'pickTurn': 5}]},
 {'teamId': 200,
  'win': 'Fail',
  'firstBlood': True,
  'firstTower': True,
  'firstInhibitor': False,
  'firstBaron': True,
  'firstDragon': False,
  'firstRiftHerald': True,
  'towerKills': 7,
  'inhibitorKills': 0,
  'baronKills': 1,
  'dragonKills': 4,
  'vilemawKills': 0,
  'riftHeraldKills': 2,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 2, 'pickTurn': 6},
   {'championId': 203, 'pickTurn': 7},
   {'championId': 89, 'pickTurn': 8},
   {'cham

In [36]:
ast.literal_eval(dataset_df['participants'][0])[0]

{'participantId': 1,
 'teamId': 100,
 'championId': 412,
 'spell1Id': 4,
 'spell2Id': 14,
 'stats': {'participantId': 1,
  'win': True,
  'item0': 3857,
  'item1': 7019,
  'item2': 3050,
  'item3': 1028,
  'item4': 3009,
  'item5': 0,
  'item6': 3364,
  'kills': 0,
  'deaths': 2,
  'assists': 29,
  'largestKillingSpree': 0,
  'largestMultiKill': 0,
  'killingSprees': 0,
  'longestTimeSpentLiving': 1023,
  'doubleKills': 0,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 33263,
  'magicDamageDealt': 20924,
  'physicalDamageDealt': 8506,
  'trueDamageDealt': 3832,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 8908,
  'magicDamageDealtToChampions': 6943,
  'physicalDamageDealtToChampions': 853,
  'trueDamageDealtToChampions': 1112,
  'totalHeal': 6909,
  'totalUnitsHealed': 5,
  'damageSelfMitigated': 28539,
  'damageDealtToObjectives': 2346,
  'damageDealtToTurrets': 1040,
  'visionScore': 94,
  'timeCCingOthers': 31,


In [37]:
ast.literal_eval(dataset_df['participantIdentities'][0])

[{'participantId': 1,
  'player': {'platformId': 'KR',
   'accountId': '-hSWoChCQOMEXum6TZe1iwT03z9ogjgdqPBjSgzif3IkGgA',
   'summonerName': 'Lehcnds',
   'summonerId': 'ATp1LH1VVMfps_1TUu3U-_wL8uavfssCI1bdqP-GOR3LtQk',
   'currentPlatformId': 'KR',
   'currentAccountId': '-hSWoChCQOMEXum6TZe1iwT03z9ogjgdqPBjSgzif3IkGgA',
   'matchHistoryUri': '/v1/stats/player_history/KR/201028421',
   'profileIcon': 4567}},
 {'participantId': 2,
  'player': {'platformId': 'KR',
   'accountId': '3la3qJ-jVirm6KUCMkTJ8rbH8eu5991N7TX_553LeT61',
   'summonerName': '지출결의',
   'summonerId': 'v_9VXJ2kGAiUDOocv56yDcptRf4myw1r8XR3mHSWiJv2HQ',
   'currentPlatformId': 'KR',
   'currentAccountId': '3la3qJ-jVirm6KUCMkTJ8rbH8eu5991N7TX_553LeT61',
   'matchHistoryUri': '/v1/stats/player_history/KR/3351083',
   'profileIcon': 3199}},
 {'participantId': 3,
  'player': {'platformId': 'KR',
   'accountId': 'BrpOIybaKU6i-J1SHu3ZKfXG9g_iNNjmUma5PgB7FRXs5iA',
   'summonerName': '니주김',
   'summonerId': 'd1hR3UJo95-fUZQpfeHZ

### 2. Preprocess Dataset

#### Extract the unique matches

In [38]:
dataset_df = dataset_df.drop_duplicates()
len(dataset_df)

3188

#### Consider only Ranked games

In [39]:
dataset_df = dataset_df[(dataset_df['queueId'] == 420)]
len(dataset_df)

2345

#### Remain the necessary information

In [54]:
dataset_df.iloc[801]

gameId                                                          5000281209
platformId                                                              KR
gameCreation                                                 1613214778257
gameDuration                                                           191
queueId                                                                420
mapId                                                                   11
seasonId                                                                13
gameVersion                                                  11.3.357.5376
gameMode                                                           CLASSIC
gameType                                                      MATCHED_GAME
teams                    [{'teamId': 100, 'win': 'Fail', 'firstBlood': ...
participants             [{'participantId': 1, 'teamId': 100, 'champion...
participantIdentities    [{'participantId': 1, 'player': {'platformId':...
Name: 982, dtype: object

In [56]:
ast.literal_eval(dataset_df['participants'][52])

[{'participantId': 1,
  'teamId': 100,
  'championId': 157,
  'spell1Id': 14,
  'spell2Id': 4,
  'stats': {'participantId': 1,
   'win': False,
   'item0': 3006,
   'item1': 1055,
   'item2': 1042,
   'item3': 1036,
   'item4': 3070,
   'item5': 0,
   'item6': 3340,
   'kills': 0,
   'deaths': 4,
   'assists': 1,
   'largestKillingSpree': 0,
   'largestMultiKill': 0,
   'killingSprees': 0,
   'longestTimeSpentLiving': 445,
   'doubleKills': 0,
   'tripleKills': 0,
   'quadraKills': 0,
   'pentaKills': 0,
   'unrealKills': 0,
   'totalDamageDealt': 16296,
   'magicDamageDealt': 1586,
   'physicalDamageDealt': 11839,
   'trueDamageDealt': 2870,
   'largestCriticalStrike': 0,
   'totalDamageDealtToChampions': 3006,
   'magicDamageDealtToChampions': 480,
   'physicalDamageDealtToChampions': 2376,
   'trueDamageDealtToChampions': 150,
   'totalHeal': 470,
   'totalUnitsHealed': 1,
   'damageSelfMitigated': 2806,
   'damageDealtToObjectives': 0,
   'damageDealtToTurrets': 0,
   'visionScore'

In [27]:
# queueId, gameVersion, teams, participants, participantIdentities
dataset_df = dataset_df[['gameVersion', 'teams', 'participants', 'participantIdentities']]
dataset_df = dataset_df.reset_index(drop=True)
dataset_df

,gameVersion,teams,participants,participantIdentities
0,11.2.353.8505,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,11.2.353.8505,"[{'teamId': 100, 'win': 'Win', 'firstBlood': F...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2,11.2.353.8505,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
3,11.2.353.8505,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
4,11.2.353.8505,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
...,...,...,...,...
2340,11.2.353.8505,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2341,11.2.353.8505,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2342,11.2.353.8505,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
2343,11.2.353.8505,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


In [28]:
dataset_df['gameDuration']

KeyError: 'gameDuration'

### 3. Split into train & test Dataset

In [10]:
div_version = '11.3.356.7268'

In [11]:
train_data = dataset_df[dataset_df['gameVersion'] < div_version].reset_index(drop=True)
test_data = dataset_df[dataset_df['gameVersion'] >= div_version].reset_index(drop=True)
print('Num train data:', len(train_data))
print('Num test data:', len(test_data))

Num train data: 11030
Num test data: 1935


#### Build dictionary with train Dataset

In [12]:
num_participants = 10
user_to_idx, idx_to_user = {}, {}
user_to_idx['UNK'] = 0
user_to_idx['MASK'] = 1
user_cnt = 2

# Build user-dictionary
for i in range(len(train_data)):
    participantIdentities = ast.literal_eval(train_data['participantIdentities'][i])
    for participant in participantIdentities:
        accountId = participant['player']['accountId']
        if accountId not in user_to_idx:
            user_to_idx[accountId] = user_cnt
            user_cnt += 1
            
for key, value in user_to_idx.items():
    idx_to_user[value] = key
    
len(user_to_idx)

19514

In [13]:
champion_to_idx, idx_to_champion = {}, {}
champion_to_idx['UNK'] = 0
champion_to_idx['MASK'] = 1
champion_cnt = 2

# Build champion-dictionary
for i in range(len(train_data)):
    participants = ast.literal_eval(train_data['participants'][i])
    for participant in participants:
        championId = participant['championId']
        if championId not in champion_to_idx:
            champion_to_idx[championId] = champion_cnt
            champion_cnt += 1
            
for key, value in champion_to_idx.items():
    idx_to_champion[value] = key
    
len(champion_to_idx)

156

In [14]:
version_to_idx, idx_to_version = {}, {}
version_to_idx['UNK'] = 0
version_cnt = 1

version_list = train_data['gameVersion'].unique()
for version in version_list:
    version_to_idx[version] = version_cnt
    version_cnt += 1

for key, value in version_to_idx.items():
    idx_to_version[value] = key
    
len(version_to_idx)

14

### 3. Create training Dataset

#### Dataset for Collaborative Filtering (=Interaction)

In [15]:
# user_champion_interaction_df (D): (N, C) matrix 
# N: number of users
# C: number of champions
# D_nc = proportion of the play

num_users = len(user_to_idx)
num_champions = len(champion_to_idx)

user_champion_matrix = np.zeros((num_users, num_champions))

In [16]:
for i in range(len(train_data)):
    participantIdentities = ast.literal_eval(train_data['participantIdentities'][i])
    participants = ast.literal_eval(train_data['participants'][i])
    for p_idx in range(num_participants):
        participant = participants[p_idx]
        participantIdentity = participantIdentities[p_idx]
        
        userIdx = user_to_idx[participantIdentity['player']['accountId']]
        championIdx = champion_to_idx[participant['championId']]
        
        user_champion_matrix[userIdx][championIdx] += 1

user_champion_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 27., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

#### Dataset for Winning Prediction (=Match)

In [48]:
# Initialize dataframe
columns = ['win', 'gameVersion']
for participant_id in range(1,num_participants+1):
    columns.append('User'+str(participant_id)+'_id')
    columns.append('User'+str(participant_id)+'_lane')
    columns.append('User'+str(participant_id)+'_champion')
    columns.append('User'+str(participant_id)+'_ban')
match_data = pd.DataFrame(index=train_data.index, columns = columns)
match_data

,win,gameVersion,User1_id,User1_lane,User1_champion,User1_ban,User2_id,User2_lane,User2_champion,User2_ban,...,User8_champion,User8_ban,User9_id,User9_lane,User9_champion,User9_ban,User10_id,User10_lane,User10_champion,User10_ban
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
version_to_idx[gameVersion]

1

### Dataset for user-BERT4Rec

In [2]:
with open('categorical_ids.pickle', 'rb') as f:
    categorical_ids = pickle.load(f)
print(len(categorical_ids['user']))
print(len(categorical_ids['champion']))

38167
158


In [6]:
with open('./match_data.pickle', 'rb') as f:
    match_data = pickle.load(f)

match_data_df = pd.concat([match_data['train'], match_data['val'], match_data['test']]).reset_index(drop=True)
num_participants = 10
#for participant_id in range(1,num_participants+1):
#    match_data_df['User'+str(participant_id)+'_stat'] = 0
match_data_df

,win,version,time,User1_id,User1_lane,User1_champion,User1_ban,User1_team,User1_stat,User2_id,...,User1_history,User2_history,User3_history,User4_history,User5_history,User6_history,User7_history,User8_history,User9_history,User10_history
0,Fail,4,1618379841203,4,7,4,43,4,"(2.5, 7.788732394366197, 0.12072434607645875, ...",5,...,0,1,2,3,4,5,6,7,8,9
1,Win,5,1618329699473,14,8,11,43,4,"(2.8333333333333335, 4.604387827317764, 0.0212...",9,...,10,11,12,13,14,15,16,17,18,19
2,Win,5,1618327333015,5,8,5,28,4,"(2.6, 4.820560747663551, 0.02118380062305296, ...",22,...,20,21,22,23,24,25,26,27,28,29
3,Fail,5,1618303705233,26,7,17,55,4,"(0.6, 5.700760777683855, 0.12595097210481826, ...",27,...,30,31,32,33,34,35,36,37,38,39
4,Fail,5,1618135389132,33,5,33,7,4,"(0.8571428571428571, 6.452348993288591, 0.0053...",34,...,40,41,42,43,44,45,46,47,48,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136306,Win,3,1618668472939,43018,8,55,43,4,"(3.0, 5.100425079702444, 0.022316684378320937,...",43053,...,1196141,1196142,0,1196143,0,0,0,0,0,0
136307,Win,3,1618665999066,43363,4,75,118,4,"(2.2857142857142856, 6.769416014449127, 0.0945...",3,...,1196144,0,0,0,0,0,0,0,0,0
136308,Fail,3,1618662279380,3,8,55,118,4,"(1.8333333333333333, 3.945414847161572, 0.0141...",3,...,0,0,0,0,0,0,1196145,1196146,0,0
136309,Fail,3,1618659499273,3,7,65,10,4,"(1.5454545454545454, 7.472644376899696, 0.1251...",3,...,0,0,1196147,0,0,0,0,1196148,0,1196149


In [26]:
stats = []
for participant_id in range(1, 11):
    stats.append(np.array([np.array(stat) for stat in match_data_df['User' + str(participant_id) + '_stat'].values]))
stats = np.concatenate(stats)

In [33]:
mu = stats.mean(axis = 0)
std = stats.std(axis= 0)
stats = (stats - mu) / std
stats = stats.reshape(10, -1, 7)
stats.shape

(10, 136311, 7)

In [45]:
new_stats = []
for participant_id in range(1,11):
    new_stats = [tuple(stat) for stat in stats[participant_id - 1]]
    match_data_df['User'+str(participant_id)+'_stat'] = new_stats

In [46]:
match_data_df['User'+str(participant_id)+'_stat']

0         (0.5084607829788115, 1.296766856189939, -0.699...
1         (-0.6592291422516439, 0.3232167647297832, 0.74...
2         (-0.002403559309512842, 0.2750097707678693, -0...
3         (1.1652863659209425, 0.4381438300669047, 0.995...
4         (0.4354801626519079, 1.8604635738920412, 0.710...
                                ...                        
136306    (-0.05105730619411514, 0.5794973532689239, -1....
136307    (-0.22134542029022308, -1.4303333630155721, -1...
136308    (-0.11187448979986796, -1.2702269630665672, -1...
136309    (-0.03889386947296444, -1.2594247516193617, -1...
136310    (-0.4402872812709335, -0.5793414954351761, 0.4...
Name: User10_stat, Length: 136311, dtype: object

In [7]:
from collections import defaultdict

num_participants = 10
UNK = 3

user_to_match_history = {}
for user_id in range(len(categorical_ids['user'])):
    user_to_match_history[user_id] = []

stat_id = 0
histories = defaultdict(list)
for match_idx in range(len(match_data_df)):
    match = match_data_df.iloc[match_idx]
    win = int(match['win'] == 'Win')
    time = match['time']
    bans = []
    for p_idx in range(num_participants):
        banIdx = match['User'+str(p_idx+1)+'_ban']
        bans.append(banIdx)
    
    for p_idx in range(num_participants):
        user =  match['User'+str(p_idx+1)+'_id']
        item = match['User'+str(p_idx+1)+'_champion']
        lane = match['User'+str(p_idx+1)+'_lane']
        # user belongs to blue-team so win should be identical
        user_stat_in_match = {}
        user_stat_in_match['id'] = stat_id
        user_stat_in_match['time'] = time
        user_stat_in_match['lane'] = lane
        user_stat_in_match['item'] = item
        user_stat_in_match['bans'] = bans
        
        if p_idx < 5:
            user_stat_in_match['win'] = win
        # user belongs to red-team so win should be reversed
        else:
            user_stat_in_match['win'] = 1-win
        if user != UNK:
            user_to_match_history[user].append(user_stat_in_match)
            histories['User'+str(p_idx+1)+'_stat'].append(stat_id)
            #match_data_df.loc[match_idx, 'User'+str(p_idx+1)+'_stat'] = stat_id
            stat_id += 1
        else:
            histories['User'+str(p_idx+1)+'_stat'].append(0)

In [14]:
for key, value in histories.items():
    match_data_df[key] = value
match_data_df

,win,version,time,User1_id,User1_lane,User1_champion,User1_ban,User1_team,User2_id,User2_lane,...,User1_stat,User2_stat,User3_stat,User4_stat,User5_stat,User6_stat,User7_stat,User8_stat,User9_stat,User10_stat
0,Fail,4,1609322998821,3,4,4,16,4,3,5,...,0,0,0,0,0,1,0,0,0,0
1,Win,4,1609238499229,3,4,14,33,4,3,7,...,0,0,0,2,0,3,0,0,0,0
2,Win,5,1612621059000,3,6,23,29,4,3,7,...,0,0,4,5,0,0,0,0,6,0
3,Fail,5,1612619789305,3,7,29,4,4,3,6,...,0,0,0,0,0,0,0,7,8,9
4,Fail,5,1612617603304,13,8,24,20,4,3,5,...,10,0,11,0,0,0,12,13,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43622,Fail,5,1613228394645,3,6,60,51,4,3,4,...,0,0,0,0,0,0,0,0,0,0
43623,Fail,5,1613226405632,3,7,55,109,4,3,4,...,0,0,0,0,0,0,0,0,0,0
43624,Fail,5,1613217326971,3,5,110,92,4,3,4,...,0,0,0,0,0,0,0,0,0,0
43625,Win,5,1613214949554,3,6,4,33,4,3,5,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# re-order the matches
for user, match_histories in user_to_match_history.items():
    user_to_match_history[user] = sorted(match_histories, key = lambda x : x['time'])

In [38]:
# check whether the item is propery ordered in sequence
for user, match_histories in user_to_match_history.items():
    cur_time = 0
    for match_history in match_histories:
        time = match_history['time']
        assert time > cur_time
        cur_time = time

In [40]:
with open('./user_history_data.pickle', 'wb') as f:
    pickle.dump(user_to_match_history, f)

In [ ]:
match_data['train'] = match_data_df[(match_data_df['time'] < args.train_end_time)].reset_index(drop=True)
match_data['val'] = match_data_df[(args.train_end_time <= match_data_df['time']) & 
                                  (match_data_df['time'] < args.val_end_time)].reset_index(drop=True)
match_data['test'] = match_data_df[(match_data_df['time'] > args.val_end_time)].reset_index(drop=True)